# Imports and Settings

In [1]:
%matplotlib inline

import csv, json, string, re, time
from random import shuffle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nltk import word_tokenize

import chainer
from chainer import Chain, Variable, Parameter
from chainer import iterators, optimizers, serializers
import chainer.initializers as I
import chainer.functions as F
import chainer.links as L

WORD_VECTOR_SIZE = 50
H_SIZE = 35
POOL_SIZE = 4
USE_GPU = False

# Read Word Vectors

In [2]:
glove = {}
f = open('glove/glove.6B.' + str(WORD_VECTOR_SIZE) + 'd.txt', 'rb')
reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
for row in reader:
    key = row[0]
    vector = map(float, row[1:])
    glove[key] = np.array(vector, dtype=np.float32).reshape(1,-1)
len(glove)

400000

# Read Dataset

In [3]:
def text2vec(text):
    tokens = word_tokenize(text.lower())
    textVec = np.array([])
    for tok in tokens:
        textVec = np.append(textVec, glove.get(tok, np.zeros((1,WORD_VECTOR_SIZE), dtype=np.float32)))
    return textVec.reshape(1, -1)

def answerpos(context, answer, answer_start):
    start = len(word_tokenize(context[:answer_start]))
    ans_len = len(word_tokenize(answer))
    
    return start, start + ans_len - 1

In [4]:
train = []
for jsonRow in json.loads(open('dataset/train.json', 'rb').read()):
    for paragraph in jsonRow['paragraphs']:
        ctxVec = text2vec(paragraph['context'])
        
        for qnaJson in paragraph['qas']:
            qnVec = text2vec(qnaJson['question'])
            
            ansStart, ansEnd = answerpos(paragraph['context'], 
                                           qnaJson['answer']['text'], 
                                           qnaJson['answer']['answer_start'])
            
            train.append((ctxVec, qnVec, ansStart, ansEnd))

len(train)

61379

In [5]:
shuffle(train)
val = train[:6000]
train = train[6000:]

print len(val), len(train)

6000 55379


In [6]:
test = []
for jsonRow in json.loads(open('dataset/test.json', 'rb').read()):
    for paragraph in jsonRow['paragraphs']:
        ctx = paragraph['context']
        ctxVec = text2vec(paragraph['context'])
        
        for qnaJson in paragraph['qas']:
            qnId = qnaJson['id']
            qnVec = text2vec(qnaJson['question'])            
            test.append((ctxVec, qnVec, qnId, ctx))
  
len(test)

36790

In [7]:
def get_batch(i, batch_size, data):
    j = min(i + batch_size, len(data))
    
    ctx = []
    qn = []
    ans_start = []
    ans_end = []
    
    cmax = 0
    qmax = 0
    for k in range(i, j):
        c, q, s, e = data[k]
        ctx.append(c)
        qn.append(q)
        ans_start.append(s)
        ans_end.append(e)
        
        cmax = max(cmax, c.shape[1])
        qmax = max(qmax, q.shape[1])
        
    cVec = np.zeros((len(ctx), cmax), dtype=np.float32)
    qVec = np.zeros((len(ctx), qmax), dtype=np.float32)        
    for i in range(len(ctx)):
        cVec[i, 0:ctx[i].shape[1]] = ctx[i]
        qVec[i, 0:qn[i].shape[1]] = qn[i]
    
    return Variable(cVec), \
           Variable(qVec), \
           Variable(np.array(ans_start, dtype=np.int32)).reshape(-1,1), \
           Variable(np.array(ans_end, dtype=np.int32)).reshape(-1,1)



In [8]:
c, q, s, e = get_batch(0, 5, train)
print c.shape
print q.shape
print s.shape
print e.shape


(5, 6850)
(5, 1050)
(5, 1)
(5, 1)


# Define Network
* RNN Tutorial: http://docs.chainer.org/en/stable/tutorial/recurrentnet.html
* Training Tutorial: http://docs.chainer.org/en/stable/tutorial/train_loop.html
* Attention: https://machinelearningmastery.com/how-does-attention-work-in-encoder-decoder-recurrent-neural-networks/
* Pointer: http://fastml.com/introduction-to-pointer-networks/

In [9]:
class CoattentionEncoder(Chain):
    def __init__(self, wordvec_size, h_size, use_gpu=False):
        super(CoattentionEncoder, self).__init__()
        
        self.h_size = h_size
        self.wordvec_size = wordvec_size
        self.use_gpu = use_gpu
        
        with self.init_scope():
            self.ctxRU = L.LSTM(wordvec_size, h_size)

            self.qnRU = L.LSTM(wordvec_size, h_size)
            self.qnLinear = L.Linear(h_size, h_size)
            
            self.outFwd = L.LSTM(3*h_size, h_size)
            self.outBwd = L.LSTM(3*h_size, h_size)
            self.outLinear = L.Linear(2*h_size, h_size)
            
            if use_gpu:
                print "CodynamicAttention uses GPU", self.use_gpu
                self.ctxRU.to_gpu()
                self.qnRU.to_gpu()
                self.qnLinear.to_gpu()
                self.outFwd.to_gpu()
                self.outBwd.to_gpu()
                self.outLinear.to_gpu()
            
    def reset_state(self):
        self.ctxRU.reset_state()
        self.qnRU.reset_state()
        self.outFwd.reset_state()
        self.outBwd.reset_state()
        
    def get_para_rep(self, para, ru):
        P = []
        for i in range(0, para.shape[1], self.wordvec_size):
            word = para[:, i:i+self.wordvec_size]
            if self.use_gpu: 
                word.to_gpu()
            P.append(ru(word))
        return F.transpose(F.dstack(P), (0, 1, 2))
            
    def __call__(self, ctx, qn):
        # context representation
        Ds = self.get_para_rep(ctx, self.ctxRU)
        
        #question representation
        Qs = self.get_para_rep(qn, self.qnRU)
        
        out_ins = []
        for i in range(Ds.shape[0]):
            D = Ds[i]
            Q = Qs[i]
            
            #attention
            affinity = F.matmul(D.T, Q)
            A_Q = F.softmax(affinity)
            A_D = F.softmax(affinity.T)

            C_Q = F.matmul(D, A_Q)
            C_D = F.matmul(F.concat((Q, C_Q), axis=0), A_D)
            
            out_ins.append(F.concat((D, C_D), axis=0).T)
        out_ins = F.transpose(F.dstack(out_ins), (0,2,1))

        #output
        h_fwd = []
        for fout in out_ins:
            h_fwd.append(self.outFwd(fout))
        h_fwd = F.dstack(h_fwd)

        h_bwd = []
        for bout in out_ins[::-1]:
            h_bwd.append(self.outBwd(bout))
        h_bwd = F.dstack(h_bwd)
        
        u_in = F.transpose(F.concat((h_fwd, h_bwd)), (0,2,1))
        U = self.outLinear(u_in.reshape(-1, 2*self.h_size))
        return U.reshape(Ds.shape[0], -1, self.h_size)

In [10]:
ctx, qn, ans_start, ans_end = get_batch(0, 5, train)

encoder = CoattentionEncoder(WORD_VECTOR_SIZE, H_SIZE)

U = encoder(ctx, qn)
print U.shape

(5, 137, 35)


In [11]:
class Highway(Chain):
    def __init__(self, h_size, pool_size, use_gpu=False):
        super(Highway, self).__init__()
        
        self.h_size = h_size
        self.pool_size = pool_size
        self.use_gpu = use_gpu
                
        with self.init_scope():
            self.MLP = L.Linear(3*h_size, h_size, nobias=True)
            self.M1 = L.Maxout(2*h_size, h_size, pool_size)
            self.M2 = L.Maxout(h_size, h_size, pool_size)
            self.M3 = L.Maxout(2*h_size, 1, pool_size)
            
            if use_gpu:
                print "Highway uses GPU", self.use_gpu
                self.MLP.to_gpu()
                self.M1.to_gpu()
                self.M2.to_gpu()
                self.M3.to_gpu()
            
    def __call__(self, U, h, us, ue):
        if self.use_gpu:
            U.to_gpu()
            h.to_gpu()
            us.to_gpu()
            ue.to_gpu()
        
        r = F.tanh(self.MLP(F.hstack([h, us, ue])))
        rs = []
        for i in range(U.shape[0]):
            rs.append(F.broadcast_to(r[i], U[i].shape))
        r = F.transpose(F.dstack(rs), (2,0,1))
        
        m_in = F.concat((U, r), axis=2).reshape(-1, 2*self.h_size)
        m1 = self.M1(m_in)
        m2 = self.M2(m1)
        m3 = self.M3(F.concat((m1,m2)))
        
        return m3.reshape(U.shape[0], -1, 1)

In [13]:
highway = Highway(H_SIZE, POOL_SIZE)

h = Variable(np.zeros((5, H_SIZE), dtype=np.float32))
us = U[:,0].reshape(5, -1)
ue = U[:,-1].reshape(5, -1)

alpha = highway(U, h, us, ue)
print alpha.shape

(5, 137, 1)


In [16]:
class DynamicPointingDecoder(Chain):
    def __init__(self, h_size, pool_size, use_gpu=False):
        super(DynamicPointingDecoder, self).__init__()
        self.use_gpu = use_gpu
                
        with self.init_scope():
            self.dec_state = L.LSTM(2*h_size, h_size)
            self.HwayStart = Highway(h_size, pool_size, use_gpu)
            self.HwayEnd = Highway(h_size, pool_size, use_gpu)
            
            if self.use_gpu:
                print "DynamicPointincDecoded uses GPU", self.use_gpu
                self.dec_state.to_gpu()
                self.HwayStart.to_gpu()
                self.HwayEnd.to_gpu()
            
    def reset_state(self):
        self.dec_state.reset_state()
            
    def __call__(self, U, us, ue):
        if self.use_gpu:
            U.to_gpu()
            us.to_gpu()
            ue.to_gpu()
        
        h = self.dec_state(F.concat((us,ue)))
        alpha = self.HwayStart(U, h, us, ue)
        s = F.argmax(alpha, axis=1).data.reshape(-1)
        beta = self.HwayEnd(U, h, U[range(U.shape[0]), s], ue)
        
        return alpha, beta

In [17]:
decoder = DynamicPointingDecoder(H_SIZE, POOL_SIZE)

alpha, beta = decoder(U, us, ue)
print alpha.shape, F.argmax(alpha, axis=1).data.reshape(1,-1)
print beta.shape, F.argmax(beta, axis=1).data.reshape(1,-1)

(5, 137, 1) [[ 46   6  78  46 105]]
(5, 137, 1) [[60 73 28 62 29]]


In [18]:
class SquadNet(Chain):
    def __init__(self, wordvec_size, h_size, pool_size, use_gpu=False):
        super(SquadNet, self).__init__()
        self.use_gpu = use_gpu
                
        with self.init_scope():
            self.encoder = CoattentionEncoder(wordvec_size, h_size, use_gpu)
            self.decoder = DynamicPointingDecoder(h_size, pool_size, use_gpu)
            
            if use_gpu:
                print "SquadNet uses GPU", self.use_gpu
                self.encoder.to_gpu()
                self.decoder.to_gpu()
            
    def reset_state(self):
        self.encoder.reset_state()
        self.decoder.reset_state()
            
    def __call__(self, ctx, qn): 
        U = self.encoder(ctx, qn)
        
        start = np.zeros(U.shape[0], 'i')
        end = np.zeros(U.shape[0], 'i') - 1        
        for i in range(4):            
            us = U[range(U.shape[0]), start]
            ue = U[range(U.shape[0]), end]
            alpha, beta = self.decoder(U, us, ue)
            
            start = F.argmax(alpha, axis=1).data.reshape(-1)
            end = F.argmax(beta, axis=1).data.reshape(-1)
        return alpha, beta

In [20]:
model = SquadNet(WORD_VECTOR_SIZE, H_SIZE, POOL_SIZE)
alpha, beta = model(ctx, qn)
print alpha.shape, F.argmax(alpha, axis=1).data.reshape(1,-1)
print beta.shape, F.argmax(beta, axis=1).data.reshape(1,-1)

(5, 137, 1) [[  5 109  26 109  52]]
(5, 137, 1) [[ 5 14 11 53 11]]


# Create Model

In [21]:
opt = optimizers.Adam(alpha=1e-3)
model = SquadNet(WORD_VECTOR_SIZE, H_SIZE, POOL_SIZE, USE_GPU)
if USE_GPU:
    model.to_gpu()
opt.setup(model)

# Define Training Loop

In [22]:
def train_model(model, opt, epoch_start, epoch_end, batch_size, print_interval):
    for epoch in range(epoch_start, epoch_end):
        print "Epoch", epoch + 1, "/", epoch_end
        startTime = time.time()
        epochScore = 0

        opt.new_epoch()
        
        interval_loss = 0
        interval_start = time.time()
        for i in range(0, len(train), batch_size):
            try:
                ctx, qn, ans_start, ans_end = get_batch(i, batch_size, train)
                if USE_GPU:
                    ans_start.to_gpu()
                    ans_end.to_gpu()

                model.reset_state()
                pred_start, pred_end = model(ctx, qn)
                
                pred_start = pred_start[:ctx.shape[0],:,:]
                pred_end = pred_end[:ctx.shape[0]]

                loss_start = F.softmax_cross_entropy(pred_start, ans_start)
                loss_end = F.softmax_cross_entropy(pred_end, ans_end)
                loss = loss_start + loss_end

                interval_loss += loss.data
                if i % print_interval == 0:
                    print i, "/", len(train), ":", \
                          interval_loss, \
                          "(" + str(time.time() - interval_start) + "s)"
                    interval_loss = 0
                    interval_start = time.time()
                
                s = F.argmax(pred_start, axis=1).data
                e = F.argmax(pred_end, axis=1).data
                for j in range(s.shape[0]):
                    if s[j] == ans_start.data[j] and e[j] == ans_end.data[j]:
                        epochScore += 1

                model.cleargrads()
                loss.backward()

                opt.update()
            except IndexError as e:
                print "Error on train index " + str(i) + ":", e
        
        valLoss = 0
        valScore = 0
        for i in range(0, len(val), batch_size):
            try:
                ctx, qn, ans_start, ans_end = get_batch(i, batch_size, val)
                if USE_GPU:
                    ans_start.to_gpu()
                    ans_end.to_gpu()

                model.reset_state()
                pred_start, pred_end = model(ctx, qn)

                loss_start = F.softmax_cross_entropy(pred_start, ans_start)
                loss_end = F.softmax_cross_entropy(pred_end, ans_end)
                valLoss += (loss_start + loss_end).data
                
                s = F.argmax(pred_start, axis=1).data
                e = F.argmax(pred_end, axis=1).data
                for j in range(s.shape[0]):
                    if s[j] == ans_start.data[j] and e[j] == ans_end.data[j]:
                        valScore += 1
            except IndexError as e:
                print "Error on val index " + str(i) + ":", e
        
        epochAcc = float(epochScore) / len(train)
        valAcc = float(valScore) / len(val)
        
        serializers.save_npz('gpu-epoch' + str(epoch+1) + '.model', model)
        print "Epoch completed in", time.time() - startTime, "seconds"
        print "Train Acc:", epochAcc, "Val Acc:", valAcc, "Val Loss:", valLoss      

# Train Model

In [23]:
train_model(model, opt, 0, 10, 100, 1000)

Epoch 1 / 10
0 / 55379 : 11.7743368149 (4.02822399139s)


KeyboardInterrupt: 

In [20]:
serializers.save_npz('gpu.model', model)

# Output Answers

In [23]:
def get_test_batch(i, batch_size, data):
    j = min(i + batch_size, len(data))
    
    ctx = []
    qn = []
    ids = []
    ctxStrs = []
    
    cmax = 0
    qmax = 0
    for k in range(i, j):
        c, q, s, e = data[k]
        ctx.append(c)
        qn.append(q)
        ids.append(s)
        ctxStrs.append(e)
        
        cmax = max(cmax, c.shape[1])
        qmax = max(qmax, q.shape[1])

    cVec = np.zeros((len(ctx), cmax), dtype=np.float32)
    qVec = np.zeros((len(ctx), qmax), dtype=np.float32)        
    for i in range(len(ctx)):
        cVec[i, 0:ctx[i].shape[1]] = ctx[i]
        qVec[i, 0:qn[i].shape[1]] = qn[i]
    
    return Variable(cVec), \
           Variable(qVec), \
           ids, \
           ctxStrs


In [24]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [26]:
test_batch_size = 100
test_print_interval = 1000

serializers.load_npz('gpu-epoch3.model', model)

f = open('pred3.csv', 'wb')
out = csv.writer(f)
out.writerow(["Id", "Answer"])

startTime = time.time()

for i in range(0, len(test), test_batch_size):
    ctx, qn, qnId, ctxStr = get_test_batch(i, test_batch_size, test)
    model.reset_state()
    start, end = model(ctx, qn)

    for j in range(len(qnId)):
        contextTokens = word_tokenize(ctxStr[j])

        s = F.argmax(start[j]).data
        e = F.argmax(end[j]).data
        
        s = min(s, len(contextTokens)-1)
        e = max(e, s)
        e = min(e, len(contextTokens)-1)        
        
        ans = ""
        for k in range(s, e + 1):
            ans += contextTokens[k] + " "
        
        out.writerow([qnId[j], normalize_answer(ans).encode('utf-8')])
    
    if i % test_print_interval == 0:
        print i, "/", len(test), "(" + str(time.time() - startTime) + "s)"
        startTime = time.time()
    
f.close()

0 / 36790 (1.40163993835s)
1000 / 36790 (14.9581708908s)
2000 / 36790 (13.9215970039s)
3000 / 36790 (13.7998709679s)
4000 / 36790 (13.6729888916s)
5000 / 36790 (14.3381428719s)
6000 / 36790 (13.3973069191s)
7000 / 36790 (12.3257830143s)
8000 / 36790 (12.3547208309s)
9000 / 36790 (12.0773820877s)
10000 / 36790 (13.3482880592s)
11000 / 36790 (12.6208658218s)
12000 / 36790 (15.8270070553s)
13000 / 36790 (12.7543549538s)
14000 / 36790 (14.1055371761s)
15000 / 36790 (13.1785640717s)
16000 / 36790 (15.712460041s)
17000 / 36790 (11.8376300335s)
18000 / 36790 (13.573777914s)
19000 / 36790 (17.5041937828s)
20000 / 36790 (13.0778388977s)
21000 / 36790 (16.0437159538s)
22000 / 36790 (13.4823410511s)
23000 / 36790 (11.2038550377s)
24000 / 36790 (13.0235090256s)
25000 / 36790 (14.7025828362s)
26000 / 36790 (13.1118619442s)
27000 / 36790 (14.967897892s)
28000 / 36790 (14.4318180084s)
29000 / 36790 (13.5588378906s)
30000 / 36790 (18.678992033s)
31000 / 36790 (12.8858289719s)
32000 / 36790 (15.9406499